In [1]:
#import dependencies 
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from collections import Counter
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.preprocessing import MinMaxScaler, Imputer
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc


from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

In [2]:
#import data
training = pd.read_csv("C:/Users/Johnny/Documents/UNH Notes and Documents 2018/Freelance/training dataset.csv", sep=',') 
scoring = pd.read_csv("C:/Users/Johnny/Documents/UNH Notes and Documents 2018/Freelance/scoring dataset.csv", sep=',') 

In [3]:
#rename 
features = training 
test_features = scoring 

In [4]:
 # Extract the ids
train_ids = features['SK_ID_CURR']
test_ids = test_features['ID']
    
# Extract the labels for training
labels = features['TARGET']

# Remove the ids and target
features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
test_features = test_features.drop(columns = ['ID'])
    

In [5]:
#encoding, folds
encoding = 'ohe'
n_folds = 5

In [9]:
if encoding == 'ohe':
    features = pd.get_dummies(features)
    test_features = pd.get_dummies(test_features)
        
    # Align the dataframes by the columns
    features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
     # No categorical indices to record
    cat_indices = 'auto'
    
# Integer label encoding
elif encoding == 'le':
        
    # Create a label encoder
    label_encoder = LabelEncoder()
        
    # List for storing categorical indices
    cat_indices = []
        
    # Iterate through each column
    for i, col in enumerate(features):
        
        if features[col].dtype == 'object':
            
            # Map the categorical features to integers
            features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
            test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

            # Record the categorical indices
            cat_indices.append(i)
    
# Catch error if label encoding scheme is not valid
else:
    raise ValueError("Encoding must be either 'ohe' or 'le'")

In [10]:
print('Training Data Shape: ', features.shape)
print('Testing Data Shape: ', test_features.shape)

Training Data Shape:  (46186, 234)
Testing Data Shape:  (3000, 234)


In [12]:
# Extract feature names
feature_names = list(features.columns)
    
# Convert to np arrays
features = np.array(features)
test_features = np.array(test_features)    

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

0        0
1        0
2        0
3        0
5        0
6        0
7        0
8        0
10       0
11       0
12       0
13       0
15       0
17       0
18       0
19       0
20       0
21       0
22       1
23       1
24       0
25       0
26       0
27       0
28       0
29       0
30       0
31       0
35       1
36       0
        ..
46152    1
46153    0
46154    0
46155    0
46156    0
46157    0
46158    0
46160    0
46161    0
46162    0
46163    0
46164    0
46165    0
46166    0
46167    0
46168    0
46169    1
46170    0
46171    0
46172    0
46173    0
46174    0
46175    0
46177    0
46178    0
46179    1
46180    0
46182    0
46183    0
46185    0
Name: TARGET, Length: 36949, dtype: int64

In [16]:
# Create the kfold object
k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
# Empty array for feature importances
feature_importance_values = np.zeros(len(feature_names))

# Lists for recording validation and training scores
valid_scores = []
train_scores = []
    
# Iterate through each fold
for train_indices, valid_indices in k_fold.split(features):
    # Training data for the fold
    train_features, train_labels = features[train_indices], labels[train_indices]
    # Validation data for the fold
    valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
    # Create the model
    model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
    # Train the model
    model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
    # Record the best iteration
    best_iteration = model.best_iteration_
        
    # Record the feature importances
    feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
    # Record the out of fold predictions
    out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
    # Record the best score
    valid_score = model.best_score_['valid']['auc']
    train_score = model.best_score_['train']['auc']
        
    valid_scores.append(valid_score)
    train_scores.append(train_score)
    # Clean up memory
    gc.enable()
    del model, train_features, valid_features
    gc.collect()
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')

Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.738073	valid's binary_logloss: 0.488717	train's auc: 0.917897	train's binary_logloss: 0.43862
Early stopping, best iteration is:
[141]	valid's auc: 0.73912	valid's binary_logloss: 0.511393	train's auc: 0.890608	train's binary_logloss: 0.473291
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[98]	valid's auc: 0.724722	valid's binary_logloss: 0.534705	train's auc: 0.861807	train's binary_logloss: 0.504907
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[80]	valid's auc: 0.748921	valid's binary_logloss: 0.545819	train's auc: 0.846583	train's binary_logloss: 0.523411
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[83]	valid's auc: 0.754539	valid's binary_logloss: 0.548716	train's auc: 0.84753	train's binary_logloss: 0.521648
Training until validation scores d

In [ ]:
#accuracy 
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

precision, recall, fscore, support = score(y_test, y_pred)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

def model(features, test_features, encoding = 'ohe', n_folds = 5):
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['ID']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['ID'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [ ]:
submission, fi, metrics = model(training, scoring)
print('Baseline metrics')
print(metrics)